In [1]:
import os
import time
import requests
import json
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from urllib.parse import urlparse
from bs4 import BeautifulSoup 
from datetime import datetime

%pip install tqdm sqlalchemy tabula-py


# Create the "tables" folder if it doesn't exist
tables_folder_path = "./tables"
if not os.path.exists(tables_folder_path):
    os.makedirs(tables_folder_path)


# Create the "matrizes" folder if it doesn't exist
pdfs_folder_path = "./courses_pdfs"
if not os.path.exists(pdfs_folder_path):
    os.makedirs(pdfs_folder_path)


Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg2
from sqlalchemy import create_engine

# Replace 'your_db_name', 'your_username', and 'your_password' with your PostgreSQL credentials
db_name = 'gradeufop_db'
username = 'postgres'
password = '12345678'

# Replace 'localhost' with the appropriate host if your PostgreSQL server is running on a different machine
host = 'localhost'
port = '5432'

# Create a connection to the PostgreSQL database
conn_str = f"postgresql://{username}:{password}@{host}:{port}/{db_name}"
engine = create_engine(conn_str)

In [3]:
import re
import unicodedata

def format_course_name(text):
    # Remove accent marks
    text = ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')
    
    # Replace symbols with a hyphen
    text = re.sub(r'[^a-zA-Z0-9]+', '-', text)
    
    # Remove leading and trailing hyphens
    text = text.strip('-')
    
    # Convert to lowercase
    text = text.lower()
    
    return text

### Busca cursos e salva os .pdfs na pasta /matrizes

In [4]:
from tqdm import tqdm

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# Set up Chrome driver service
chromedriver_path = (
    "./chromedriver"  # Replace with the path to your chromedriver executable
)
service = Service(chromedriver_path)

# Set up Chrome driver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navigate to the URL
url = "https://www.escolha.ufop.br/cursos"
driver.get(url)

# Wait for the page to load
time.sleep(2)

# Find elements with class "ufop-glossary-row"
elements = driver.find_elements(By.CLASS_NAME, "ufop-glossary-row")

# Extract the href links from child anchor 'a' tags
links = []
courses_dict = {"id": [], "name": []}
for element in elements:
    link_element = element.find_element(By.TAG_NAME, "a")
    href = link_element.get_attribute("href")
    links.append(href)

    courses_dict["name"].append(link_element.text)
    courses_dict["id"].append(format_course_name(link_element.text))


indexes_to_delete = []
course_pdfs = []

# Navigate to each link and download PDF files
print("Buscando os links .pdf ...")
with tqdm(total=len(links), desc="Progresso", ascii=True) as pbar:
    for i, link in enumerate(links):
        driver.get(link)

        matriz_elements = driver.find_elements(
            By.CLASS_NAME, "field-name-field-matriz-curricular"
        )

        for element in matriz_elements:
            link_elements = element.find_elements(By.TAG_NAME, "a")

            if len(link_elements) == 1:
                href = link_elements[0].get_attribute("href")
                course_pdfs.append({"course": courses_dict["id"][i], "link": href})
                continue

            for link_element in link_elements:
                href = link_element.get_attribute("href")
                course_type = link_element.text
                course_name = f"{courses_dict['name'][i]} ({course_type})"
                course_id = format_course_name(course_name)
                courses_dict["name"].append(course_name)
                courses_dict["id"].append(format_course_name(course_name))

                course_pdfs.append({"course": course_id, "link": href})
            indexes_to_delete.append(i)

        pbar.set_postfix(curso=f"{courses_dict['name'][i]}")
        pbar.update(1)

courses_dict["id"] = [
    item for i, item in enumerate(courses_dict["id"]) if i not in indexes_to_delete
]
courses_dict["name"] = [
    item for i, item in enumerate(courses_dict["name"]) if i not in indexes_to_delete
]

valid_pdf_substrings = [".pdf", "codCurso="]
for pdf in course_pdfs:
    import os

for pdf in course_pdfs:
    if any(text in pdf["link"] for text in valid_pdf_substrings):
        response = requests.get(pdf["link"])
        parsed_url = urlparse(pdf["link"])
        filename = f"{pdf['course']}.pdf"
        file_path = os.path.join(pdfs_folder_path, filename)

        # Check if the file already exists in the folder
        if not os.path.exists(file_path):
            with open(file_path, "wb") as file:
                file.write(response.content)
            print(f"{filename} salvo com sucesso.")
        else:
            print(f"{filename} já existe no diretório. Pulando...")


# Quit the driver
driver.quit()

courses_df = pd.DataFrame(courses_dict).sort_values(by='id', ascending = True)
courses_df['created_at'] = datetime.now()
courses_df.to_csv(f"{tables_folder_path}/course.csv", index=False)

# Save courses table in database
courses_df.to_sql('course', engine, index=False, if_exists='replace')


Buscando os links .pdf ...


Progresso:  16%|#5        | 7/44 [00:09<00:48,  1.31s/it, curso=Ciência e Tecnologia de Alimentos]


KeyboardInterrupt: 

In [25]:
import uuid

URL = "https://zeppelin10.ufop.br/HorarioAulas/"

desired_departments = ["DECSI", "DECEA", "DEELT", "DEENP", "DEETE"]
semester = "23.1"


def get_HTML_content(URL, department):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(URL)
    elem = driver.find_element(By.XPATH, "//*[text()='{}']".format(department))
    elem.click()
    URL = driver.current_url
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, "lxml")
    driver.quit()
    return soup


def parse_schedule_string(schedule_string):
    entries = []
    
    if schedule_string == '':
        return entries
    
    schedule_parts = schedule_string.split(" / ")
    
    if len(schedule_parts) == 0:
        schedule_parts.append(schedule_string)

    for part in schedule_parts:
        day, time_info = part.split(" ")
        start_time, end_time = time_info.split("-")
        class_type = end_time[-2]  # T for theoretical, P for practical
        end_time = end_time[:-3]  # Remove the class type from end_time

        entry = {
            "day_of_week": day,
            "start_time": start_time,
            "end_time": end_time,
            "class_type": class_type,
        }
        entries.append(entry)

    return entries


def get_field_list(html_content, field):
    field_list = []
    table = html_content.find("table", {"id": "formPrincipal:tabela"})
    if table:
        tbody = table.find("tbody")
        tr_elements = tbody.find_all("tr")

        for i, tr in enumerate(tr_elements):
            if field == "descricao":
                span = tr.find(
                    "span", {"id": "formPrincipal:tabela:{}:{}".format(i, "disciplina")}
                )
                title = span.find_parent("a").get(
                    "title"
                )  # Extract the 'title' attribute of the parent <a> tag
                field_list.append(title)
                continue

            span = tr.find(
                "span", {"id": "formPrincipal:tabela:{}:{}".format(i, field)}
            )
            field_list.append(span.text)

    return field_list


def get_departments():
    r = requests.get(URL)
    departments_table = []
    departments_list = []
    soup = BeautifulSoup(r.text, "html.parser")  # Use 'html.parser' as the parser

    # Find the table with the specified id
    table = soup.find("table", {"id": "formPrincipal:tabela"})
    if table:
        tbody = table.find("tbody")

        # Find all <tr> elements within <tbody>
        tr_elements = tbody.find_all("tr")

        for i, tr in enumerate(tr_elements):
            tableCode = tr.find(
                "span", {"id": "formPrincipal:tabela:{}:codigoDepartamento".format(i)}
            )
            tableName = tr.find(
                "span", {"id": "formPrincipal:tabela:{}:descricao".format(i)}
            )
            # if tableCode and tableCode.text.strip() in desired_departments:
            #     departments_table.append(
            #         {"id": tableCode.text.strip(), "name": tableName.text.strip()}
            #     )
            #     departments_list.append(tableCode.text.strip())
            departments_table.append(
                {"id": tableCode.text.strip(), "name": tableName.text.strip()}
            )
            departments_list.append(tableCode.text.strip())

        departments_df = pd.DataFrame(departments_table)
        departments_df["created_at"] = datetime.now()
        return departments_df, departments_list


def get_discipline_tables(departments_list):
    discipline_dict = {"id": [], "name": [], "description": [], "department_id": []}
    class_dict = {
        "id": [],
        "class_number": [],
        "discipline_id": [],
        "professor": [],
    }
    schedule_dict = {
        "id": [],
        "discipline_class_id": [],
        "day_of_week": [],
        "start_time": [],
        "end_time": [],
        "class_type": [],
    }

    for department in departments_list:
        if department not in desired_departments:
            continue

        print(f"Buscando {department}")
        html_content = get_HTML_content(URL, department)
        columns_list = [
            "codigo",
            "disciplina",
            "descricao",
            "turma",
            "horario",
            "professores",
        ]
        columns_dict_list = {}

        for column_name in columns_list:
            field = get_field_list(html_content, column_name)
            columns_dict_list[column_name] = field

        for i in range(len(columns_dict_list["codigo"])):
            discipline_dict["id"].append(columns_dict_list["codigo"][i])
            discipline_dict["name"].append(columns_dict_list["disciplina"][i])
            discipline_dict["description"].append(columns_dict_list["descricao"][i])
            discipline_dict["department_id"].append(department)

            discipline_class_id = str(uuid.uuid4())
            class_dict["id"].append(discipline_class_id)
            class_dict["class_number"].append(columns_dict_list["turma"][i])
            class_dict["discipline_id"].append(columns_dict_list["codigo"][i])
            class_dict["professor"].append(columns_dict_list["professores"][i])

            schedule_entries = parse_schedule_string(columns_dict_list["horario"][i])
            for entry in schedule_entries:
                schedule_dict["id"].append(str(uuid.uuid4()))
                schedule_dict["discipline_class_id"].append(discipline_class_id)
                schedule_dict["day_of_week"].append(entry["day_of_week"])
                schedule_dict["start_time"].append(entry["start_time"])
                schedule_dict["end_time"].append(entry["end_time"])
                schedule_dict["class_type"].append(entry["class_type"])

    discipline_df = pd.DataFrame(discipline_dict).drop_duplicates()
    class_df = pd.DataFrame(class_dict)
    schedule_df = pd.DataFrame(schedule_dict)

    currentTime = datetime.now()

    discipline_df["created_at"] = currentTime
    class_df["created_at"] = currentTime
    class_df["semester"] = semester
    schedule_df["created_at"] = currentTime

    return discipline_df, class_df, schedule_df


department_df, departments_list = get_departments()
department_df.to_csv(f"{tables_folder_path}/department.csv", index=False)
department_df.to_sql("department", engine, index=False, if_exists="replace")

(
    discipline_df,
    discipline_class_df,
    discipline_class_schedule_df,
) = get_discipline_tables(departments_list)
discipline_df.to_csv(f"{tables_folder_path}/discipline.csv", index=False)
discipline_df.to_sql("discipline", engine, index=False, if_exists="replace")

discipline_class_df.to_csv(f"{tables_folder_path}/discipline_class.csv", index=False)
discipline_class_df.to_sql("discipline_class", engine, index=False, if_exists="replace")

discipline_class_schedule_df.to_csv(
    f"{tables_folder_path}/discipline_class_schedule.csv", index=False
)
discipline_class_schedule_df.to_sql(
    "discipline_class_schedule", engine, index=False, if_exists="replace"
)

Buscando DECEA


In [ ]:
import tabula
import pandas as pd
import numpy as np
import uuid

import re

code_pattern = r"[A-Z]{3}\d{3}"
subject_pattern = r"\b[A-Z]+\b"
classes_pattern = r"^(T P|T|P)$"
prerequisite_pattern = r"[A-Z]{3}\d{3}|\d+\s+horas"
chs_che_pattern = r"^\d+\/\d+$"


def get_col_idx(df_value, pattern):
    indexes = []
    for i, item in enumerate(df_value):
        if i == 0:
            continue
        match = re.search(pattern, str(item), re.UNICODE)
        if match is not None:
            indexes.append(i)

    return indexes


def replace_carriage_return(arr):
    series = pd.Series(arr)

    # Replace '\r' with an empty string in the Series values
    series = series.astype(str).str.replace(r"\r", "")

    return series


def get_prerequisites(df_value):
    prerequisites = []
    prereq_idx = get_col_idx(df_value, prerequisite_pattern)
    if len(prereq_idx) > 0:
        prerequisites = [df_value[i] for i in prereq_idx]

    return format_prerequisites(prerequisites)


def get_discipline(df_value):
    subject_idx = get_col_idx(df_value, subject_pattern)
    if len(subject_idx) > 0:
        return re.sub(code_pattern, "", df_value[subject_idx[0]])
    return ""


def get_chs_che(df_value):
    chs_che_idx = get_col_idx(df_value, chs_che_pattern)

    if len(chs_che_idx) > 0:
        chs_che_list = df_value[chs_che_idx[0]].split("/")
        return tuple(map(int, chs_che_list))

    return ("", "")


def get_classes(df_value, classes_idx):
    classes = []
    if len(classes_idx) > 0:
        classes = [df_value[i] for i in classes_idx]
        return " ".join(classes)

    return ""


def get_period(df_value, eletiva):
    if eletiva or not df_value[0]:
        return ""

    for item in reversed(df_value):
        if item:
            return int(item)
    return ""


def get_period2(df_value, mandatory):
    if not mandatory or not df_value[0]:
        return ""

    for item in reversed(df_value):
        if item:
            return int(item)
    return ""


def get_cha(df_value, chs):
    if not chs:
        return ""

    chs_alt = chs * 1.2

    if len(df_value) == 0:
        return ""

    for text in df_value:
        try:
            formatted_text = int(text)
            if formatted_text == chs or formatted_text == chs_alt:
                return formatted_text
        except ValueError:
            continue

    return ""


def format_prerequisites(df_value):
    if len(df_value) == 0:
        return ""

    requisites = []
    for text in df_value:
        if not text:
            continue

        matches = re.findall(code_pattern, text)
        requisites.extend(matches)

    joined_matches = " ".join(requisites)
    return joined_matches


def get_formatted_df(df, eletiva):
    df = df.replace({r"\r": " "}, regex=True)
    df_struct = {
        "codigo": [],
        "disciplina": [],
        "prerequisitos": [],
        "chs": [],
        "che": [],
        "cha": [],
        "aulas": [],
        "periodo": [],
    }
    ideal_df = pd.DataFrame(data=df_struct)

    ideal_columns = ideal_df.columns.to_list()

    column_names = df.columns.tolist()
    disc_indexes = [i for i, item in enumerate(column_names) if "DISCIPLINAS" in item]

    iterIdx = -1
    classes_idx = []

    for idx, value in enumerate(df.values):
        if idx == 0:
            iterIdx = -1
            classes_idx = get_col_idx(value, classes_pattern)
            ideal_df.at[0, ideal_columns[0]] = ""
            ideal_df.at[0, ideal_columns[1]] = ""
            ideal_df.at[0, ideal_columns[2]] = ""
            ideal_df.at[0, ideal_columns[3]] = ""
            ideal_df.at[0, ideal_columns[4]] = ""
            ideal_df.at[0, ideal_columns[5]] = ""
            ideal_df.at[0, ideal_columns[6]] = ""
            ideal_df.at[0, ideal_columns[7]] = ""
            continue

        chs, che = get_chs_che(value)
        ideal_df.at[idx, ideal_columns[0]] = value[0]
        ideal_df.at[idx, ideal_columns[1]] = get_discipline(value)
        ideal_df.at[idx, ideal_columns[2]] = get_prerequisites(value)
        ideal_df.at[idx, ideal_columns[3]] = chs
        ideal_df.at[idx, ideal_columns[4]] = che
        ideal_df.at[idx, ideal_columns[5]] = get_cha(value, chs)
        ideal_df.at[idx, ideal_columns[6]] = get_classes(value, classes_idx)
        ideal_df.at[idx, ideal_columns[7]] = get_period(value, eletiva)

        if value[0] != "":
            iterIdx = -1
            continue

        if iterIdx == -1:
            iterIdx = idx - 1

        ideal_df.loc[iterIdx, :] = [
            f"{item1} {item2}".strip()
            for item1, item2 in zip(ideal_df.values[iterIdx], ideal_df.values[idx])
        ]

    ideal_df.drop(ideal_df[ideal_df["codigo"] == ""].index, inplace=True)
    ideal_df[["chs", "che", "cha"]] = ideal_df[["chs", "che", "cha"]].astype(int)
    ideal_df["eletiva"] = eletiva

    return ideal_df


def get_prerequisite_df(discipline_course_df):
    prerequisite_df = pd.concat(
        discipline_course_df.apply(create_prerequisite_rows, axis=1).tolist(),
        ignore_index=True,
    )

    prerequisite_df["created_at"] = datetime.now()
    return prerequisite_df


def create_prerequisite_rows(row):
    prerequisites = row["prerequisites"].split()
    if len(prerequisites):
        return pd.DataFrame(
            {
                "id": [str(uuid.uuid4()) for _ in range(len(prerequisites))],
                "discipline_course_id": [row["id"] for _ in range(len(prerequisites))],
                "prerequisite_discipline_id": prerequisites,
            }
        )
    else:
        return pd.DataFrame()


def get_discipline_course_tables(df, course_id, mandatory):
    df = df.replace({r"\r": " "}, regex=True)
    df_struct = {"id": [], "discipline_id": [], "period": [], "prerequisites": []}
    discipline_course_df = pd.DataFrame(data=df_struct)

    ideal_columns = discipline_course_df.columns.to_list()

    column_names = df.columns.tolist()
    disc_indexes = [i for i, item in enumerate(column_names) if "DISCIPLINAS" in item]

    iterIdx = -1

    for idx, value in enumerate(df.values):
        if idx == 0:
            iterIdx = -1
            discipline_course_df.at[0, "id"] = ""
            discipline_course_df.at[0, "discipline_id"] = ""
            discipline_course_df.at[0, "period"] = ""
            discipline_course_df.at[0, "prerequisites"] = ""
            continue

        discipline_course_df.at[idx, "id"] = str(uuid.uuid4())
        discipline_course_df.at[idx, "discipline_id"] = value[0]
        discipline_course_df.at[idx, "period"] = get_period2(value, mandatory)
        discipline_course_df.at[idx, "prerequisites"] = get_prerequisites(value)

        if value[0] != "":
            iterIdx = -1
            continue

        if iterIdx == -1:
            iterIdx = idx - 1

        discipline_course_df.loc[iterIdx, :] = [
            f"{item1} {item2}".strip()
            for item1, item2 in zip(
                discipline_course_df.values[iterIdx], discipline_course_df.values[idx]
            )
        ]

    discipline_course_df.drop(
        discipline_course_df[discipline_course_df["discipline_id"] == ""].index,
        inplace=True,
    )
    discipline_course_df["mandatory"] = mandatory
    discipline_course_df["course_id"] = course_id
    prerequisite_df = pd.DataFrame()
    if not discipline_course_df.empty:
        prerequisite_df = get_prerequisite_df(discipline_course_df)

    discipline_course_df.drop("prerequisites", axis=1)

    return discipline_course_df, prerequisite_df


def save_table_to_csv(df):
    # Save the DataFrame to a CSV file
    filename = "table.csv"
    df.to_csv(filename, index=False)
    print(f"Table saved as {filename}")


def save_table_to_json(df):
    # Save the DataFrame to a CSV file
    filename = "table.json"
    df.to_json(filename, orient="records")
    print(f"Table saved as {filename}")


def scrape_table_from_pdf(pdf_path):
    # Read the table from the PDF file
    df_list = tabula.read_pdf(pdf_path, pages="all")

    course_id = os.path.splitext(os.path.basename(pdf_path))[0]

    discipline_course_dfs = []
    prerequisite_dfs = []

    for df in df_list:
        df.fillna("", inplace=True)
        # Filter and select the desired columns based on the header
        header = df.columns.to_list()
        eletivas_column = next(
            (col for col in header if "DISCIPLINAS ELETIVAS" in col), None
        )
        obrigatorias_column = next(
            (col for col in header if "DISCIPLINAS OBRIGATÓRIAS" in col), None
        )

        if "DISCIPLINAS OBRIGATÓRIAS" in header:
            discipline_course_df, prerequiste_df = get_discipline_course_tables(
                df, course_id, True
            )
            discipline_course_dfs.append(discipline_course_df)
            prerequisite_dfs.append(prerequiste_df)

        elif (
            "DISCIPLINAS ELETIVAS" in header
            or "DISCIPLINAS ELETIVAS PRÉ-REQUISITO" in header
        ):
            discipline_course_df, prerequiste_df = get_discipline_course_tables(
                df, course_id, False
            )
            discipline_course_dfs.append(discipline_course_df)
            prerequisite_dfs.append(prerequiste_df)
        else:
            continue

    combined_discipline_course_df = pd.concat(discipline_course_dfs, ignore_index=True)

    combined_prerequisite_df = pd.concat(prerequisite_dfs, ignore_index=True)
    discipline_course_df = combined_discipline_course_df[
        combined_discipline_course_df["discipline_id"].isin(discipline_df["id"])
    ]

    prerequisite_df = combined_prerequisite_df
    if not combined_prerequisite_df.empty:
        prerequisite_df = combined_prerequisite_df[
            combined_prerequisite_df["discipline_course_id"].isin(
                discipline_course_df["id"]
            )
        ]

    return discipline_course_df, prerequisite_df


files = os.listdir(pdfs_folder_path)
pdf_files = [
    os.path.join(pdfs_folder_path, file) for file in files if file.endswith(".pdf")
]

for pdf_file in [
    "./courses_pdfs/engenharia-de-computacao.pdf",
    "./courses_pdfs/engenharia-de-producao-jm.pdf",
    "./courses_pdfs/sistemas-de-informacao.pdf",
    "./courses_pdfs/engenharia-eletrica.pdf",
]:
    print(f"Buscando {pdf_file}")
    discipline_course_df, prerequisite_df = scrape_table_from_pdf(pdf_file)
    discipline_course_df.to_csv(
        f"{tables_folder_path}/discipline_course.csv",
        mode="a",
        header=not os.path.exists(f"{tables_folder_path}/discipline_course.csv"),
        index=False,
    )
    discipline_course_df.to_sql(
        "discipline_course", con=engine, if_exists="append", index=False
    )

    prerequisite_df.to_csv(
        f"{tables_folder_path}/prerequisite.csv",
        mode="a",
        header=not os.path.exists(f"{tables_folder_path}/prerequisite.csv"),
        index=False,
    )
    prerequisite_df.to_sql("prerequisite", con=engine, if_exists="replace", index=False)

# discipline_course_df, prerequisite_df = scrape_table_from_pdf(
#     "./courses_pdfs/fisica-bacharelado.pdf"
# )

Buscando ./courses_pdfs/engenharia-de-computacao.pdf


Got stderr: jul 24, 2023 5:57:13 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'
jul 24, 2023 5:57:14 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'
jul 24, 2023 5:57:14 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'
jul 24, 2023 5:57:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'
jul 24, 2023 5:57:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'



Buscando ./courses_pdfs/engenharia-de-producao-jm.pdf


Got stderr: jul 24, 2023 5:57:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'
jul 24, 2023 5:57:16 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'
jul 24, 2023 5:57:16 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'
jul 24, 2023 5:57:16 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'
jul 24, 2023 5:57:16 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'



Buscando ./courses_pdfs/sistemas-de-informacao.pdf


Got stderr: jul 24, 2023 5:57:17 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'
jul 24, 2023 5:57:18 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'
jul 24, 2023 5:57:18 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'



Buscando ./courses_pdfs/engenharia-eletrica.pdf


Got stderr: jul 24, 2023 5:57:18 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'
jul 24, 2023 5:57:19 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'
jul 24, 2023 5:57:20 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'
jul 24, 2023 5:57:20 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans-Bold' for 'Arial-Black'



In [ ]:
files = os.listdir(pdfs_folder_path)
pdf_files = [
    os.path.join(pdfs_folder_path, file) for file in files if file.endswith(".pdf")
]
print(len(pdf_files))

52
